In [10]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import time
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
import numpy as np
from scipy.cluster import *
import word2vec

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from IPython.display import clear_output
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

In [11]:
FILEmodelForInfinitive = Model.load('udpipe_syntagrus.model')
#model = word2vec.load('dataForNeurotext/withOutTypeModel.txt')

In [19]:
%%time
class wordsClass (object):
    model = ''
    vectorOfStr = np.zeros(2000)
    centroids = torch.zeros(2000,300)
    ClasterOfWord = {}
    nomberKnownOfWords = 0
    nomberUNKnownOfWords = 0
    modelForInfinitive = ''

    def loadCentroidsFromTXT (self, source):
        self.centroids = torch.from_numpy(np.loadtxt(source, delimiter=' ', dtype=float))
        self.centroids = torch.reshape(self.centroids, (2000, 300)).cuda()
        
    def saveCentroidsToTXT (self, source):
        f = open (source, 'w')
        for i in range (2000):
            for j in range (300):
                f.write(str(self.centroids[i][j]) + " ")
        f.close()
        
    def saveDict (self, source):
        f = open(source, 'w')
        for k, v in self.ClasterOfWord.items():
            f.write(str(k) + " " + str(v) + "\n")
        f.close()
            
    def loadDict (self, source):
        f = open(source, 'r')
        for strings in f:
            strings = strings.split(' ')
            self.ClasterOfWord.update({strings[0]: int(strings[1])})
        f.close()
        
    def generateCentroids (self, nomberOfClasters, NomberOfIter):
        #scipy.cluster.vq.kmeans2(model.vectors, 2000, iter=100)
        self.centroids = vq.kmeans(self.model.vectors, nomberOfClasters, iter=NomberOfIter)[0]
        
    def generateDict (self, source): #источник = txt для model
        f = open (source, 'r')
        n=0
        words = []
        for line in f:
            line = line.split()
            words.append(line[0])
            n = n+1
        print (words[1])
        f.close()
        for word in log_progress(words, every=100):
            distToCentroids = torch.sum(((self.centroids - self.model[word])**2),1)
            relationID = int(torch.argmin(distToCentroids, 0))
            ClasterOfWord.update({word: relationID})
    
    def distanceBetween2words (self, word1, word2):
        word1 = word1.lower()
        word2 = word2.lower()
        try:
            len = model.distance(tag_ud(text=word1), tag_ud(text=word2))
        except Exception:
            try:
                len =  model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2))
            except Exception:
                try:
                    len =  model.distance(tag_ud(text=word1), tag_ud(text=word2.capitalize()))
                except Exception:
                    len = model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2.capitalize()))
        return len
    
    def coordinate (self, word):
        word = word.lower()
        try:
            coord = self.model[tag_ud(text=word)]
        except Exception:
            try:
                coord =  self.model[tag_ud(text=word.capitalize())]
            except Exception:
                coord = "error"
        return torch.from_numpy (coord)
    
    def getKey(self, value):
        for k, v in self.ClasterOfWord.items():
            if int(v) == value:
                print (k)
        return k
    
    def addNewWordInClaster (self, newWord, baseWord):
        self.ClasterOfWord.update({newWord: self.ClasterOfWord[baseWord]})
    
    def tryToUpgradeSelfWord2vec (self, word):
        try:
            #print (word)
            relationID = self.ClasterOfWord[word.lower()]
            self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
            self.nomberKnownOfWords = self.nomberKnownOfWords + 1
            #print (relationID)
        except Exception:
            if not re.search(r'[\W]', word):
                #print(self.wordToInf(text=word))
                relationID = self.ClasterOfWord[self.wordToInf(text=word)]
                self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
                self.nomberKnownOfWords = self.nomberKnownOfWords + 1
                #print (relationID)
            else: 
                raise
                
            
    def text2Vec (self, text):
        self.vectorOfStr = np.zeros(2000)
        words = re.findall(r'[0-9A-Za-zА-Яа-я-.]+', text)
        for word in words:
            try :
                self.tryToUpgradeSelfWord2vec(word)
            except Exception:
                for singleWord in re.findall(r"[\w']+", word):
                    try: 
                        self.tryToUpgradeSelfWord2vec(singleWord)
                    except Exception:
                        self.nomberUNKnownOfWords = self.nomberUNKnownOfWords + 1
                        #print ("the word is new: " + singleWord)
        self.vectorOfStr = self.vectorOfStr / self.nomberKnownOfWords    
        return self.vectorOfStr
    
    def wordToInf(self, text):
        process_pipeline = Pipeline(self.modelForInfinitive, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        wordInfo = process_pipeline.process(text).split('\n')[4].split('\t')
        if (wordInfo[3] == 'NUM'):
            return ('_NUM_' + ('x' * len(wordInfo[2])))
        else:
            return wordInfo[2]
        
#проработать встречи ::
#реализовать работу с реляционной БД (?):
#(текст->оценка принадлежности вектор->оценка принадлежности)
#поддержка новый слов через сервис

wordsClass = wordsClass()
wordsClass.modelForInfinitive = FILEmodelForInfinitive
wordsClass.loadDict('correct_dictionary.txt')

#wordsClass.text2Vec("рaссказывать")
#wordsClass.ClasterOfWord['Alibaba']
#wordsClass.saveDict('dict words has clasters.txt')

#wordsClass.text2Vec("Китайский интернет-гигант Alibaba меняет бизнес-модель торговой площадки AliExpress, дав возможность продавцам из России, а также Турции, Италии и Испании продавать на ней свои товары. Об этом сообщает «Интерфакс».")
print (wordsClass.vectorOfStr)

[0. 0. 0. ... 0. 0. 0.]
CPU times: user 230 ms, sys: 17.2 ms, total: 247 ms
Wall time: 264 ms


In [13]:
NOfData = 1848
input_size = 1929
num_classes = 42

In [45]:
relationTranslate = {
    0: 'авто/мото',
    1: 'активный отдых',
    2: 'бизнес',
    3: 'домашние животные',
    4: 'здоровье',
    5: 'знакомство и общение',
    6: 'игры',
    7: 'ИТ (компьютеры и софт)',
    8: 'кино',
    9: 'красота и мода',
    10: 'кулинария',
    11: 'культура и искусство',
    12: 'литература',
    13: 'мобильная связь и интернет',
    14: 'музыка',
    15: 'наука и техника',
    16: 'недвижимость',
    17: 'новости и СМИ',
    18: 'безопасность',
    19: 'образование',
    20: 'обустройство и ремонт',
    21: 'политика',
    22: 'продукты питания',
    23: 'промышленность',
    24: 'путешествия',
    25: 'работа',
    26: 'развлечения',
    27: 'религия',
    28: 'дом и семья',
    29: 'спорт',
    30: 'страхование',
    31: 'телевидение',
    32: 'товары и услуги',
    33: 'увлечения и хобби',
    34: 'финансы',
    35: 'фото',
    36: 'эзотерика',
    37: 'электроника и бытовая техника',
    38: 'эротика',
    39: 'юмор',
    40: 'общество, гуманитарные науки',
    41: 'дизайн и графика'
}

In [46]:
badIndexes = np.zeros(2000)
idexes = [58, 81,130,150,178,259,291,301,321,330,335,365,373,418,453,470,478,494,543,602,629,645,666,690,725,
746,770,775,821,825,826,827,844,855,939,942,945,968,1011,1015,1040,1052,1083,1095,1111,1132,1205,1210,1302,
1305,1314,1366,1371,1382,1393,1497,1511,1531,1535,1569,1571,1613,1649,1727,1740,1766,1807,1823,1837,1900,1914]
for inde in idexes:
    badIndexes[inde] = 1

In [14]:
%%time
#загрузка векторов и формирование list[vecOfText, vecOfRelation]

vecOfTexts = torch.zeros(input_size)
vecOfResult = torch.zeros(num_classes)
strings = open('data.txt' ,'r').read().split ('\n')
vec = []
for i  in range (NOfData):
    i = 0
    j = 0
    k = 0
    string = strings[i].split('|')
    
    for coordOfText in string[0].split('~'):
        vecOfTexts[i] = 10000 * float(coordOfText)
        i = i + 1

    for coordOfResult in string[1].split('~'):
        vecOfResult[j] = float(coordOfResult)
        j = j + 1

    vec.append([vecOfTexts, vecOfResult])
    k = k + 1

CPU times: user 30.5 s, sys: 93.3 ms, total: 30.6 s
Wall time: 30.8 s


In [15]:
#разделене на тренировочные и тестовые данные
vec_train = []
vec_test = []
for i in range (NOfData):
    if (i < 1700):
        vec_train.append(vec [i])
    else:
        vec_test.append(vec [i])

In [54]:
%%time
#тренеруемся

hidden_size = 100      
num_epochs = 20         
batch_size = 400       
learning_rate = 0.001  

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(vec):   #vec_train
        images = Variable(images.view(-1, input_size))         
        labels = Variable(labels.view(-1, num_classes))

        optimizer.zero_grad()                             
        outputs = net(images)                             
        loss = criterion(outputs, labels)                 
        loss.backward()                                   
        optimizer.step()                                  
        
        if (i+1) % 100 == 0:                              
            clear_output()
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, (i)//100, len(vec_train)//batch_size, loss.data.item()))


Epoch [20/20], Step [17/4], Loss: 0.0003
CPU times: user 2min 46s, sys: 39.2 s, total: 3min 25s
Wall time: 1min 3s


In [55]:
testText = 'Подготовка к школе вместе с "Полиглот"! В школе начинается английский? Мы подготовим! Приглашаем детей дошкольного возраста и первоклассников на летний интенсив-курс по английскому языку!   Курс рассчитан на обучение английскому детей 6-7 лет и включает в себя азы языка - алфавит, основы чтения и базовую лексику. Благодаря летнему курсу ребенку будет намного легче осваивать новый предмет в школе и адаптироваться к новой социальной среде! Основы грамматики, навыки чтения, усвоенные на уроках, позволят максимально комфортно влиться в школьные занятия. Хотите привить своему ребенку любовь и интерес к новому языку? Приходите, мы точно знаем как это сделать!  Летний интенсив рассчитан на 18 академических часов и займет полтора месяца. Он включает в себя три модуля:'
wordsClass.text2Vec(testText)
vecOfTheText = torch.zeros(input_size)
k = 0
for i in range (2000):
    if (badIndexes[i]==0):
        vecOfTheText[k] = wordsClass.vectorOfStr[i]
        k = k + 1

data = Variable(vecOfTheText.view(-1, input_size))
outputs = net(data)
for i in range (42):
    if (float (outputs[0][i]) > 0):
        print (relationTranslate[i] + ':\t\t' + str(float (outputs[0][i])))
#print (outputs)

авто/мото:		0.00781187042593956
активный отдых:		0.014087893068790436
знакомство и общение:		0.6646062135696411
игры:		0.024317476898431778
ИТ (компьютеры и софт):		0.027786174789071083
красота и мода:		0.0009706267155706882
литература:		0.6894176006317139
мобильная связь и интернет:		0.010921396315097809
наука и техника:		0.00718187028542161
недвижимость:		0.017370369285345078
новости и СМИ:		0.018632175400853157
безопасность:		0.008340239524841309
обустройство и ремонт:		0.019627729430794716
политика:		0.0019035923760384321
путешествия:		0.024162745103240013
религия:		0.007814777083694935
дом и семья:		0.003380067180842161
товары и услуги:		0.023995239287614822
увлечения и хобби:		0.02548847533762455
эзотерика:		0.015736497938632965
электроника и бытовая техника:		0.010894142091274261
